# Mount Google Drive

Mount your google drive to be used for storing the storing the dataset into it (optional to save datasets on mega to it) and also for saving the results in the drive.

In [1]:
from google.colab import drive
drive.mount._DEBUG = False
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Move Dataset from Mega to Drive

Move the file with the provided link (Provide your username and password if it's not a public link) to your google drive already mounted before in the previous step. 

### Install Mega

In [2]:
import sys, os, urllib.request
import time
import subprocess
import contextlib
from IPython.display import clear_output
from functools import wraps
import errno
import os
import signal
import subprocess
import shlex
import glob

HOME = os.path.expanduser("~")
if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    runSh,
    loadingAn,
)

if not os.path.exists("/usr/bin/mega-cmd"):
    loadingAn()
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
    runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
    print("MEGA is installed.")
    clear_output()


### provide URL and Mega ID

Add the URL to fetch from Mega, and the username and password for those files if it wasn't a public URL and to use the pro mega quota if you have a pro account.

In [3]:
USERNAME = ""  
PASSWORD = "" 
TAGGED_DATASET_URL = "https://mega.nz/file/Ec5V1DKK#AwsUvEbDe-izExphB4SYRS9bP2LtyGBftVJM9AJO_Wg"
OTHER_DATASET_URL  = "https://mega.nz/file/MNo0ABSB#8rDqevxdQmtaNFKjQ3RS9v2pF-jL8xzmM9LLxIfAhG0"
OUTPUT_PATH = ""

In [4]:
# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']

def latest_file(folder):
  list_of_files = glob.glob(f'{folder}/*') # * means all 
  latest_file = max(list_of_files, key=os.path.getctime)
  return latest_file

def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out


def transfare(url):
    import codecs
    decoder = codecs.getincrementaldecoder("UTF-8")()
    cmd = ["mega-get", url, OUTPUT_PATH]
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        print(line)
        
if not OUTPUT_PATH:
  os.makedirs("downloads", exist_ok=True)
  OUTPUT_PATH = "downloads"


class TimeoutError(Exception):
    pass

def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wraps(func)(wrapper)

    return decorator


@timeout(10)
def runShT(args):
    return runSh(args, output=True)

def login(): 
    runShT(f"mega-login {USERNAME} {PASSWORD}")
    
if not (USERNAME == "" or PASSWORD == ""):
    try:
        login()
    except TimeoutError:
        runSh('mega-whoami', output=True)
else:
    print("Please Input your Mega IDs.")

transfare(TAGGED_DATASET_URL)
tagged_dataset_path = latest_file('./downloads')
transfare(OTHER_DATASET_URL)
other_dataset_path = latest_file('./downloads')

Please Input your Mega IDs.
[Initiating MEGAcmd server in background. Log: /root/.megaCmd/megacmdserver.log]
TRANSFERRING ||#........................................||(1/1236 MB:   0.14 %)  
TRANSFERRING ||#........................................||(2/1236 MB:   0.20 %)  
TRANSFERRING ||#........................................||(3/1236 MB:   0.29 %)  
TRANSFERRING ||#........................................||(5/1236 MB:   0.43 %)  
TRANSFERRING ||#........................................||(7/1236 MB:   0.62 %)  
TRANSFERRING ||#.......................................||(11/1236 MB:   0.90 %)  
TRANSFERRING ||#.......................................||(16/1236 MB:   1.31 %)  
TRANSFERRING ||#.......................................||(23/1236 MB:   1.89 %)  
TRANSFERRING ||#.......................................||(29/1236 MB:   2.35 %)  
TRANSFERRING ||##......................................||(36/1236 MB:   2.93 %)  
TRANSFERRING ||##......................................||(42/1236 MB:  

# Fetch & Clone Repo

Clone the `image-tagging-tools` repo as it has all the required utils and codes from preprocessing the image datasets to training the models and using them to classify the images. 

In [5]:
!git clone https://github.com/kk-digital/image-tagging-tools.git

Cloning into 'image-tagging-tools'...
remote: Enumerating objects: 501, done.
remote: Counting objects: 100% (225/225), done.
remote: Compressing objects: 100% (148/148), done.
remote: Total 501 (delta 124), reused 152 (delta 76), pack-reused 276
Receiving objects: 100% (501/501), 6.67 MiB | 14.32 MiB/s, done.
Resolving deltas: 100% (217/217), done.


# Preprocess the dataset images (Stage 1)

Use the `image-dataset-processor` utils from the previously cloned repo to process a directory of images (paths to directory of images or an archived dataset), and computes the images metadata along with its CLIP embeddings and writes the result into a JSON file into `output_folder`

### Install requirements 

In [6]:
%%capture
%pip install ascii_graph open_clip_torch patool fire

### Import the module/Utility 

In [15]:
import sys
sys.path.insert(0, './image-tagging-tools/image-dataset-processor/')
sys.path.insert(0, './image-tagging-tools/')
from ImageDatasetProcessor import ImageDatasetProcessor
from classify import main as classify_main
from train import main as train_main
from classify_helper_functions import clean_file
import patoolib
import shutil


def is_archive(path: str) -> bool:
    """method to check if a given path is an archive.
    :param path: The file path to check. 
    :type path: str
    :returns: `True` if the given path is a path of an archived file. 
    :rtype: bool
    """
    
    try: 
        patoolib.get_archive_format(path)
        return True 
    except Exception: 
        return False 

def unzip_folder(folder_path :str):
    """takes an archived file path and unzip it.
    :param folder_path: path to the archived file.
    :type folder_path: str
    :returns: path of the new exracted folder 
    :rtype: str
    """
    dir_path  = os.path.dirname(folder_path)
    file_name = os.path.basename(folder_path).split('.zip')[0]
    os.makedirs(dir_path , exist_ok=True)
    
    print("[INFO] Extracting the archived file...")
    patoolib.extract_archive(folder_path, outdir=dir_path)
    print("[INFO] Extraction completed.")
    
    return latest_file(dir_path)

def clean_directory(
                    dir_path : str ,
                    only_sub_dir : bool = False
                    ):
    """ clean a directory files and folders

    :param dir_path: path to the directory which will be cleaned.
    :type dir_path: str
    :param only_sub_dir: an option to make it only sub directories 
                          for ex: in cleaning pixel-art-tagged folder.
    :type only_sub_dir: bool
    :rtype: None
    """
    for dir in os.listdir(dir_path):
      sub_dir = os.path.join(dir_path, dir)
      
      if os.path.isfile(sub_dir): # if it's a file then clean the file  
        if only_sub_dir: # no subfiles allowed for example in the pixel-art-tagged 
          os.remove(sub_dir)
          print(f"[Removing] {sub_dir}")
          continue 
        clean_file(sub_dir)
        continue

      if len(os.listdir(sub_dir)) == 0: # Empty folder, delte it 
        shutil.rmtree(sub_dir)
        #os.system(f'rm -r {sub_dir}') 
        print(f'[Removing] {sub_dir}')
        continue

      if os.path.isdir(sub_dir) and only_sub_dir: # move to the sub-directory and clean it.
        clean_directory(sub_dir)
      else:
        shutil.rmtree(sub_dir)  


### Unzipping and cleaning for the tagged images

In [16]:
if is_archive(tagged_dataset_path):
  tagged_images_folder = unzip_folder(tagged_dataset_path)
  clean_directory(tagged_images_folder, only_sub_dir=True)

[INFO] Extracting the archived file...
patool: Extracting ./downloads/pixel-art-tagged-v2-merged-2022-11-22.zip ...
patool: running /usr/bin/7z x -o./downloads -- ./downloads/pixel-art-tagged-v2-merged-2022-11-22.zip
patool: ... ./downloads/pixel-art-tagged-v2-merged-2022-11-22.zip extracted to `./downloads'.
[INFO] Extraction completed.
[Removing] ./downloads/pixel-art-tagged-merged-2022-11-22/not-pixel-art/00006-32x32_100k_patches-mirror-auto1.zip
[Warning] problem with ./downloads/pixel-art-tagged-merged-2022-11-22/other-validation/https___i.pinimg.com_originals_00_92_c4_0092c473456acd2e4819da18ac964760.gif, cannot identify image file './downloads/pixel-art-tagged-merged-2022-11-22/other-validation/https___i.pinimg.com_originals_00_92_c4_0092c473456acd2e4819da18ac964760.gif'
[Warning] removing ./downloads/pixel-art-tagged-merged-2022-11-22/other-validation/https___i.pinimg.com_originals_00_92_c4_0092c473456acd2e4819da18ac964760.gif
[Removing] ./downloads/pixel-art-tagged-merged-2022

### set required variables by the utility

Initialize the required parameters needed by the dataset preprocessor utility and they are described as follows: 

* `input_folder` _[str]_ -  path to the directory containing sub-folders of each tag.
* `output_folder` _[str]_ - path to the directory where to save the files into it.

* `clip_model` _[str]_ - CLIP model to be used

* `pretrained` _[str]_ - the pre-trained model to be used for CLIP
* `batch_size` _[int]_ -  number of images to process at a time
* `num_threads` _[int]_ - the number to be used in this process

* `device` _[str]_ -  the device to be used in computing the CLIP embeddings, if `None` is provided then `cuda` will be used if available


In [17]:
dataset_path = tagged_images_folder
output_folder = "preprocess_output"
clip_model = "ViT-B-32"
pretrained = 'openai'
batch_size = 32
num_threads = 4
device = None

### Run the Preprocessor

In [18]:
ImageDatasetProcessor.process_dataset(
    dataset_path, 
    output_folder, 
    clip_model, 
    pretrained,
    batch_size, 
    num_threads, 
    device
)

100%|███████████████████████████████████████| 354M/354M [00:05<00:00, 63.7MiB/s]


['pos-environment-architecture', 'not-pixel-art', 'pos-style-synth-wave', 'pos-color-pallete', 'pos-pixel-art-scifi', 'pos-environmental-architecture', 'pos-pixel-art-character', 'other-validation', 'pos-environment', 'pos-pixel-art-tutorial', 'pos-pixel-art-environmental', 'pos-video-game-top-perspective', 'pos-video-game-isometric', 'pos-pixel-art-bad', 'pos-scifi', 'pos-environment-space', 'pos-pixel-art', 'pos-video-game-map', 'pos-pixel-art-best', 'pos-video-game-3d', 'not-pixel-art-digital', 'pos-pixel-art-texture', 'pos-style-comic', 'pos-sprite-sheet', 'pos-waifus', 'pos-character', 'pos-video-game', 'pos-environmental-natural', 'pos-video-game-flat', 'pos-machines', 'pos-video-game-style-miltech-kag', 'pos-environment-natural', 'pos-pixel-art-architecture', 'not-pixel-art-real-photo', 'pos-character-portrait', 'pos-video-game-side-scrolling', 'pos-video-game-top-down', 'pos-environmental-space', 'pos-style-tile-based', 'other-training', 'pos-pixel-art-space']
[INFO] dataset pi

### Train script variables


* `metadata_json` _[string]_ - _[required]_ - The path to the metadata json file. 
* `tag_to_hash_json` _[string]_ - _[required]_ - The path to tag-to-hash json file. 

* `output` _[string]_ - _[optional]_ - The path to the output directory.
* `test_per` _[float]_ - _[optional]_ - The percentage of the test images from the dataset, default = 0.1 


In [19]:
metadata_json = './preprocess_output/input-metadata.json' 
tag_to_hash_json = './preprocess_output/input-tag-to-image-hash-list.json'
output_dir = './models_reports'
test_per = 0.1

### Run training script

In [20]:
train_main(
    metadata_json = metadata_json,
    tag_to_hash_json = tag_to_hash_json,
    output_dir = output_dir,
    test_per = test_per
)

[INFO] Finished.


### Classification script variables

* `directory` _[string]_ - _[required]_ - The path to the images' folder or images' .zip file. 
* `metadata_json` _[string]_ - _[required]_ - The path to the metadata json file for CLIP embeddings. 
* `output` _[string]_ - _[optional]_ - The path to the output directory for the inference results. 
* `model` _[string]_ - _[optional]_ - The path to the models' .pkl files directory or single .pkl file model.
* `output_bins` _[int]_ - _[optional]_ -  The number of bins of the results for each model.


#### Classification fot other-validation folder (pre-computed CLIP embeddings)

In [21]:
folder_path    = os.path.join(tagged_images_folder,"other-validation")
output_dir     = "./classification_other_validation"
json_file_path =  "/content/preprocess_output/input-metadata.json"
bins_number    = 10
model_path     = "./models_reports/models" 

In [22]:
classify_main(
        folder_path    = folder_path , 
        output_dir     = output_dir, 
        json_file_path = json_file_path, 
        bins_number    = bins_number , 
        model_path     = model_path, 
        )

[INFO] Output folder ./classification_other_validation


100%|██████████| 220/220 [00:31<00:00,  6.95it/s]


[INFO] Finished.


#### Classifction for .zip file full of images (takes more time)

In [46]:
folder_path    = other_dataset_path
output_dir     = "./classification_other_folder"
json_file_path =  "/content/preprocess_output/input-metadata.json"
bins_number    = 10
model_path     = "./models_reports/models" 

In [ ]:
classify_main(
        folder_path    = folder_path , 
        output_dir     = output_dir, 
        json_file_path = json_file_path, 
        bins_number    = bins_number , 
        model_path     = model_path, 
        )